In [ ]:
pip install pytrends

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import re
import os
from pandas.io.json import json_normalize
from datetime import date,datetime,timedelta
import time
def categoria(cats):
    switcher={
        '7':'Finance',    
        '18':'Shopping',
        '37': 'Banking',
        '279': 'Credit & Lending',
        '468': 'Auto Financing',
        '811': 'Credit Cards',
        '38': 'Insurance',
        '107': 'Investing',
        '44':'Beauty & Fitness',
        '8':'Games',
        '45':'Health',
        '65':'Hobbies & Leisure',
        '299':'Online Communities',
        '14':'People & Society',
        '20':'Sports',
        '0': 'Global',
        '35': 'Music & Audio',
        '71': 'Food & Drink',
        '560' :'Non-Alcoholic Beverages',
        '442': 'Ecology & Environment',
        '1307': 'Recycling',
        '1349': 'Water Supply & Treatment',
        '3': 'Arts & Entertainment',
        '34': 'Movies',
        '36': 'TV & Video',
        '185': 'Fashion & Style',
        '22': 'Books & Literature',
        '799': 'Business Education',
        '5': 'Computers & Electronics',
        '899': 'Game Systems & Consoles',
        '1290': 'Family-Oriented Games & Activities',
        '105': 'Online Games',
        '195': 'Reproductive Health',
        '257': 'Substance Abuse',
        '66': 'Pets & Animals',
        '977': 'Special Occasions',
        '11': 'Home & Garden',
        '1131': 'Family & Relationships',
        '154': 'Kids & Teens',
        '70': 'Gifts & Special Event Items',
        '67': 'Travel'
        }
    return switcher.get(cats,"Categoria Invalida")
interese = pd.DataFrame(columns=('dates','keyword','interes', 'hour','dateq'))
regions = pd.DataFrame(columns=('dates','keyword','interes', 'hour','location'))
big_trend_brand = pd.DataFrame(columns=('query','interes','keyword','nquery','hour'))

pytrend = TrendReq(timeout=(45),retries=2)
now = datetime.now()- timedelta(hours=5)
hour = now.strftime('%H:%M')
day = now.strftime('%Y-%m-%d')
proyecto = 'HSBC - Día del Padre' # El nombre del proyecto se debe ir cambiando y representa la marca para la que se trabaja.
#periodos = ['2017-01-01 2017-12-31','2018-01-01 2018-12-31','2019-01-01 2019-12-31','2020-01-01 2020-12-31','2021-01-01 2021-12-31']
#periodos = ['2021-04-01 2021-04-30','2021-05-01 2021-05-31','2021-11-01 2021-11-30', '2022-01-01 2022-02-15']
periodos = ['2019-06-01 2019-06-30', '2020-06-01 2020-06-30', '2021-06-01 2021-06-30', '2022-01-01 2022-04-30']
# El periodo se visualiza por AÑO / MES / DIA, de punto "A" a punto "B".

#Búsquedas desde una lista en un archivo csv. se carga de la siguiente forma:

#dfnombres=pd.read_csv('Prod HS.csv')
#dfnombres['Producto']=dfnombres['Producto']. apply(lambda x : x.strip())
#dfnombres['Producto']=dfnombres['Producto']. apply(lambda x : x.split(' ')[0])
#solonombres=dfnombres.drop_duplicates(subset=['Producto'])
#categorias = ['0'] #COVID
#groups = [solonombres['Producto'][i:i + 5] for i in range(0, len(solonombres), 5)] #Para correr de 5 en 5
#groups = [solonombres['Producto'][i:i] for i in range(0, len(solonombres), 1)] #Para correr de 1 en 1


#Búsquedas desde una lista personalizada:

#ListaKeywords  = ['Play Doh','Fud','Leche Entera','Zwan','Santa Clara', 'Deslactosada', 'Alpura', 'Lala', 'Leche', 'Jamón', 'Leche Lala', 'Lala 100', 'Alpura 100']
ListaKeywords  = ['dia del padre','dia del papa', 'mi papa', 'ser padre']
categorias = ['0'] #COVID
groups = [ListaKeywords[i:i + 5] for i in range(0, len(ListaKeywords), 5)]

for kw_list in groups:
    for period in periodos:
        pytrend.build_payload(kw_list = kw_list, timeframe = period,geo='MX')
        #pytrend.build_payload(kw_list = kw_list, timeframe = period,geo='US')
        interest_over_time_df = pytrend.interest_over_time()
        interes = pd.DataFrame(columns=('dates','keyword','interes', 'hour','dateq','marca'))
        for kw in kw_list:
            interes.dates = interest_over_time_df.index
            interes.keyword = kw
            interes.interes = interest_over_time_df[kw].values
            interes.hour = hour
            interes.dateq = day
            interes.marca = proyecto
            interese = pd.concat([interese,interes], axis=0, ignore_index=True)
        interest_by_region_df = pytrend.interest_by_region(inc_low_vol=True, inc_geo_code=False)

        #
        state = pd.DataFrame(columns=('dates','keyword','interes', 'hour','location','marca'))
        for kw in kw_list:
            state.interes = interest_by_region_df[kw].values
            state.location = interest_by_region_df.index
            state.hour = hour
            state.dates = day
            state.keyword = kw
            state.marca = proyecto
            regions = pd.concat([regions,state], axis=0, ignore_index=True)
        #
        for cats in categorias:
            #REALIZA LA CONSULTA A GOOGLE TRENDS CON LOS PARAMETROS ESTABLECIDOS
            pytrend.build_payload(kw_list,geo = 'MX', timeframe = period, cat = cats)
            #pytrend.build_payload(kw_list,geo = 'US', timeframe = period, cat = cats)
            trends= pytrend.related_queries()
            #DEVUELVE UN JSON {"KEYWORD":{"TOP:QUERY,VALUE" , "RISIGN:QUERY,VALUE"}}
            data1 = pd.DataFrame(columns=('query','value','keyword'))
            data2 = pd.DataFrame(columns=('query','value','keyword'))

            for kw in kw_list:  
                datat = pd.DataFrame(trends[kw]['top'])
                datar = pd.DataFrame(trends[kw]['rising'])
                if datat.empty is False or datar.empty is False: 
                    datat['keyword'] = kw
                    datar['keyword'] = kw
                    data1 = pd.concat([data1,datat], axis=0, ignore_index=True, sort=False)
                    data2 = pd.concat([data2,datar], axis=0, ignore_index=True, sort=False)

            data1 = data1.rename(columns={'value':'interes'})
            data2 = data2.rename(columns={'value':'nquery'})

            trend_brand = pd.merge(data1, data2, how='outer', on='query')
            trend_brand.keyword_x.fillna(trend_brand.keyword_y,  inplace=True)
            trend_brand = trend_brand.rename(columns={'keyword_x':'keyword'})
            trend_brand = trend_brand.fillna(0)
            trend_brand = trend_brand.drop(['keyword_y'], axis=1)

            trend_brand['hour'] = hour
            trend_brand['dates'] = day
            trend_brand['periodo'] = period
            trend_brand['categoria'] = categoria(cats)
            trend_brand['marca'] = proyecto
            namef = 'GT_KW'+categoria(cats)+period+'.csv'
            big_trend_brand = big_trend_brand.append(trend_brand, ignore_index = True)
            ###importacion CSV
            #export_csv = trend_brand.to_csv (namef, index = None, header=True , encoding='UTF-8')
            print(categoria(cats),kw_list)
            print(period)
            print(hour)

                ###importacion a ADW
                #ntable = "STG_BS_GT_BRANDS"
                #data = trend_brand.astype(str)
                #colus = ', '.join([':'+str(x+1) for x in range(len(data.columns.tolist()))])
                #sql='insert into '+ntable+' values('+colus+')'
                #print(sql)

                #df_list = data.values.tolist()
                #n = 0
                #for i in data.iterrows():
                #    cursor.execute(sql,df_list[n])
                #    connection.commit()
                #    n += 1

            time.sleep(10)

    #big_trend_brand  
    #export_csv = trend_brand.to_csv (namef, index = None, header=True , encoding='UTF-8')

Global ['dia del padre', 'dia del papa', 'mi papa', 'ser padre']
2019-06-01 2019-06-30
08:20
Global ['dia del padre', 'dia del papa', 'mi papa', 'ser padre']
2020-06-01 2020-06-30
08:20
Global ['dia del padre', 'dia del papa', 'mi papa', 'ser padre']
2021-06-01 2021-06-30
08:20
Global ['dia del padre', 'dia del papa', 'mi papa', 'ser padre']
2022-01-01 2022-04-30
08:20


In [ ]:
interese

,dates,keyword,interes,hour,dateq,marca
0,2019-06-01,dia del padre,4,13:48,2022-05-02,HSBC - Día del Padre
1,2019-06-02,dia del padre,5,13:48,2022-05-02,HSBC - Día del Padre
2,2019-06-03,dia del padre,5,13:48,2022-05-02,HSBC - Día del Padre
3,2019-06-04,dia del padre,5,13:48,2022-05-02,HSBC - Día del Padre
4,2019-06-05,dia del padre,5,13:48,2022-05-02,HSBC - Día del Padre
...,...,...,...,...,...,...
835,2022-04-26,ser padre,2,13:48,2022-05-02,HSBC - Día del Padre
836,2022-04-27,ser padre,2,13:48,2022-05-02,HSBC - Día del Padre
837,2022-04-28,ser padre,1,13:48,2022-05-02,HSBC - Día del Padre
838,2022-04-29,ser padre,1,13:48,2022-05-02,HSBC - Día del Padre


In [ ]:
regions

,dates,keyword,interes,hour,location,marca
0,2022-05-02,dia del padre,85,13:48,Aguascalientes,HSBC - Día del Padre
1,2022-05-02,dia del padre,88,13:48,Baja California,HSBC - Día del Padre
2,2022-05-02,dia del padre,91,13:48,Baja California Sur,HSBC - Día del Padre
3,2022-05-02,dia del padre,93,13:48,Campeche,HSBC - Día del Padre
4,2022-05-02,dia del padre,88,13:48,Chiapas,HSBC - Día del Padre
...,...,...,...,...,...,...
507,2022-05-02,ser padre,7,13:48,Tamaulipas,HSBC - Día del Padre
508,2022-05-02,ser padre,10,13:48,Tlaxcala,HSBC - Día del Padre
509,2022-05-02,ser padre,7,13:48,Veracruz,HSBC - Día del Padre
510,2022-05-02,ser padre,9,13:48,Yucatan,HSBC - Día del Padre


In [ ]:
big_trend_brand

,query,interes,keyword,nquery,hour,dates,periodo,categoria,marca
0,el dia del padre,100,dia del padre,0,13:48,2022-05-02,2019-06-01 2019-06-30,Global,HSBC - Día del Padre
1,feliz dia del padre,61,dia del padre,1100,13:48,2022-05-02,2019-06-01 2019-06-30,Global,HSBC - Día del Padre
2,feliz dia del padre,61,dia del padre,255400,13:48,2022-05-02,2019-06-01 2019-06-30,Global,HSBC - Día del Padre
3,feliz dia del padre,61,dia del padre,600,13:48,2022-05-02,2019-06-01 2019-06-30,Global,HSBC - Día del Padre
4,feliz dia del padre,17,dia del papa,1100,13:48,2022-05-02,2019-06-01 2019-06-30,Global,HSBC - Día del Padre
...,...,...,...,...,...,...,...,...,...
463,dias festivos 2022,0,dia del padre,350,13:48,2022-05-02,2022-01-01 2022-04-30,Global,HSBC - Día del Padre
464,cuando es el día del padre 2022,0,dia del padre,200,13:48,2022-05-02,2022-01-01 2022-04-30,Global,HSBC - Día del Padre
465,cuando es el dia del padre cdmx,0,dia del padre,140,13:48,2022-05-02,2022-01-01 2022-04-30,Global,HSBC - Día del Padre
466,carrera dia del padre 2022,0,dia del padre,140,13:48,2022-05-02,2022-01-01 2022-04-30,Global,HSBC - Día del Padre


In [ ]:
interese.to_csv('{}_Intereses_{}_{}.csv'.format(proyecto, day, hour), encoding='utf-8')

regions.to_csv('{}_Regiones_{}_{}.csv'.format(proyecto, day, hour), encoding='utf-8')

big_trend_brand.to_csv('{}_Related_{}_{}.csv'.format(proyecto, day, hour), encoding='utf-8')

In [ ]:
pwd

'/Users/raul.castillo/SCRIPTS'

In [ ]:
groups = [kw_list[i:i + 5] for i in range(0, len(kw_list), 5)]
groups

[['Play Doh', 'plastilina patra niños', 'plastilina no toxica']]

In [ ]:
pytrend.trending_searches(pn='mexico')

,0
0,Ezra Miller
1,WhatsApp Web
2,SAT
3,WhatsApp
4,Raquel Pankowsky
5,Maps
6,SCJN
7,Tenencia CDMX
8,CFE
9,Portugal vs Macedonia


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=705432f3-5a59-4b7f-9d5a-243608199265' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>